# <div align="center" style="color: #ff5733;">SIL Disbursement DownPayment Details 2024</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """select a.digitalLoanAccountId, a.loanAccountNumber, a.new_loan_type, a.mobileno,
a.disbursementDateTime,
 a.disbursedLoanAmount, a.disbursedLoanTenur,a.downPaymentAmount
, (coalesce(cast(c.price as float64),0)) as Items_Sold_Price
, case when (coalesce(cast(c.price as float64),0)) = 0 then 0 else round(cast(coalesce(a.downPaymentAmount, '0') as numeric) /(coalesce(cast(c.price as float64),0)), 2) end Downpayment_rate
, a.purpleKey
, a.merchantName
, asf.STORE_TAG
from `risk_credit_mis.loan_master_table`  a
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category c 
    on a.digitalLoanAccountId = c.digitalLoanAccountId
left join `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf on upper(asf.PURPLE_KEY) = upper(a.purpleKey)
where upper(new_loan_type) like '%SIL%'
and disbursementDateTime is not null 
and format_date('%Y', disbursementDateTime) = '2024'"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 2a460afc-2346-4fd2-9491-8e749e69d666 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
print(f"The shape of the dataframe df is :\t {df.shape}")

The shape of the dataframe df is :	 (101629, 13)


In [5]:
df.to_csv(r"SIL2024DisbursementLoan_Downpayment.csv", index = False)

In [6]:
df.head()

,digitalLoanAccountId,loanAccountNumber,new_loan_type,mobileno,disbursementDateTime,disbursedLoanAmount,disbursedLoanTenur,downPaymentAmount,Items_Sold_Price,Downpayment_rate,purpleKey,merchantName,STORE_TAG
0,720563c4-3e71-4016-976f-2d5b563f0180,60826256170019,SIL-Instore,639604564596,2024-07-01 16:22:26,13299.0,6,5800.00,19099.0,0.30,MFRL8YN,FC Home Center Alapan,Appliances
1,cb55864c-40da-471f-a7d2-2dbaf724d54e,60824856540012,SIL-Instore,639184882880,2024-04-28 09:52:27,10114.0,6,20000.00,30114.0,0.66,M06WAWY,Home Along Bagong Silang 1,Appliances
2,ed2f1d46-c6d2-4e95-a091-6a395b76d55c,60823611090011,SIL-Instore,639265262881,2024-01-07 10:37:58,8239.0,9,2060.00,10299.0,0.20,MY7RZCA,Home Along Baliuag 2,Appliances
3,98c98986-d48d-42c5-a37e-15eff0c71700,60824858150011,SIL-Instore,639952664427,2024-04-28 10:59:16,8063.0,6,896.00,8959.0,0.10,M2Q79KY,Home Along Los Banos 2,Appliances
4,c47e9d9d-7f85-471e-b1a3-fbb3fe3888e6,60827313590015,SIL-Instore,639466847850,2024-08-07 18:39:06,12599.0,6,5400.00,17999.0,0.30,M8ZJKRN,Home Along Meycauayan 1 Sto Nino,Appliances


In [7]:
sq = """select * from prj-prod-dataplatform.dl_loans_db_raw.tdbk_segment_mtb;"""

dfd = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 63ab3714-7c28-44c2-8fbe-51752e2a124e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [8]:
sq = """Select distinct lmt.digitalLoanAccountId, lmt.purpleKey,
lmt.startApplyDateTime,
  tbtc.loanCategoryId, tbpm.product_id, tqpd.segment_id, tsm.segment_name
, lmt.disbursementDateTime, lmt.new_loan_type
from `risk_credit_mis.loan_master_table` lmt 
inner join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc on tbtc.digitalLoanAccountId = lmt.digitalLoanAccountId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm on tbpm.product_id = tbtc.loanCategoryId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_qr_promoter_details tqpd on tqpd.digitalLoanAccountId = lmt.digitalLoanAccountId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_segment_mtb tsm on cast(tsm.segment_id as numeric) = cast(tqpd.segment_id as numeric)
where tsm.segment_name is not null
;"""

In [9]:
dfd = client.query(sq).to_dataframe()

In [10]:
duplicates = dfd[dfd.duplicated(subset='digitalLoanAccountId', keep=False)]
duplicates

,digitalLoanAccountId,purpleKey,startApplyDateTime,loanCategoryId,product_id,segment_id,segment_name,disbursementDateTime,new_loan_type
0,adc83c43-c767-4d06-baee-06a24414ba43,MI9KLNA,2024-08-15 13:08:41,1747,1747,8,Premium,NaT,SIL-Instore
2,79e72177-e5e0-46d6-ba7d-146b5083e821,M0JY2TK,2024-06-20 15:12:07,1747,1747,8,Premium,2024-06-20 15:20:09,SIL-Instore
8,9e6e5483-6bbc-4c25-b7c2-e6d9f94a0d2e,MWE2YZF,2024-08-26 17:47:40,1747,1747,8,Premium,2024-08-26 18:07:29,SIL-Instore
11,dfcfa520-10ed-420c-8b73-94e7c2df7425,MUFDX4A,2024-08-11 11:14:13,1731,1731,8,Premium,2024-08-11 11:23:19,SIL-Instore
13,d21bb327-fce4-442b-8f1b-c2467b9521cc,MWZPTIY,2024-07-25 16:17:27,1731,1731,8,Premium,NaT,SIL-Instore
...,...,...,...,...,...,...,...,...,...
137020,47f0b2d2-5c1f-4bb1-b209-7ca5fadb2f70,M80SH64,2024-08-04 19:21:43,1731,1731,10,SIL-Zero,2024-08-04 19:31:17,SIL ZERO
137024,f842872b-b044-4bf8-9000-cc89ba0c3368,MN9OHTI,2024-08-16 14:02:53,1747,1747,10,SIL-Zero,NaT,SIL ZERO
137025,9efee7b5-92dd-4e5f-8c60-304085383c7b,MO4LQKN,2024-08-12 15:36:29,1747,1747,10,SIL-Zero,NaT,SIL ZERO
137026,69a62dd7-4bbc-4285-b6c0-150ef2542260,MB69X86,2024-08-11 16:54:53,1731,1731,10,SIL-Zero,NaT,SIL ZERO


In [11]:
dfd[dfd['digitalLoanAccountId'] == '79e72177-e5e0-46d6-ba7d-146b5083e821']

,digitalLoanAccountId,purpleKey,startApplyDateTime,loanCategoryId,product_id,segment_id,segment_name,disbursementDateTime,new_loan_type
2,79e72177-e5e0-46d6-ba7d-146b5083e821,M0JY2TK,2024-06-20 15:12:07,1747,1747,8,Premium,2024-06-20 15:20:09,SIL-Instore
323,79e72177-e5e0-46d6-ba7d-146b5083e821,M0JY2TK,2024-06-20 15:12:07,1731,1731,8,Premium,2024-06-20 15:20:09,SIL-Instore


In [12]:
# Group the DataFrame by 'digitalLoanAccountId' and 'purpleKey'
grouped_df = dfd.groupby(['digitalLoanAccountId', 'purpleKey'])

# Calculate the maximum 'segment_name' for each group
max_segment_name = grouped_df['segment_name'].max()

# Create a new DataFrame with the grouped index and calculated maximum values
result_df = pd.DataFrame(max_segment_name).reset_index()

# Pivot the DataFrame to have 'segment_name' in both index and columns
pivoted_df = result_df.pivot(index=['digitalLoanAccountId', 'purpleKey'], columns='segment_name', values='segment_name')

# Fill missing values with 'null'
pivoted_df.fillna('null', inplace=True)

pivoted_df = pivoted_df.reset_index()

In [13]:
pivoted_df.head()

segment_name,digitalLoanAccountId,purpleKey,Advanced,Basic,Premium,SIL-Zero,Starter
0,00000ad2-697b-49a4-9f16-b177eb3b5f39,M1ZYHKU,null,Basic,null,null,null
1,000093c0-1c86-4efb-b648-5b34580ee28e,MQ4BDJV,null,Basic,null,null,null
2,0000a6c8-2d70-466a-99a4-2be1fe5d5e0b,MLAYROX,null,null,null,SIL-Zero,null
3,00013637-7faf-4589-8644-54762724b9ed,MB69X86,Advanced,null,null,null,null
4,0002339c-70ff-40c2-a1ad-f922c97760c9,M8A59CP,Advanced,null,null,null,null


In [14]:
df = df.merge(pivoted_df, on='digitalLoanAccountId', how = 'left')

In [15]:
df.head()

,digitalLoanAccountId,loanAccountNumber,new_loan_type,mobileno,disbursementDateTime,disbursedLoanAmount,disbursedLoanTenur,downPaymentAmount,Items_Sold_Price,Downpayment_rate,purpleKey_x,merchantName,STORE_TAG,purpleKey_y,Advanced,Basic,Premium,SIL-Zero,Starter
0,720563c4-3e71-4016-976f-2d5b563f0180,60826256170019,SIL-Instore,639604564596,2024-07-01 16:22:26,13299.0,6,5800.00,19099.0,0.30,MFRL8YN,FC Home Center Alapan,Appliances,MFRL8YN,Advanced,null,null,null,null
1,cb55864c-40da-471f-a7d2-2dbaf724d54e,60824856540012,SIL-Instore,639184882880,2024-04-28 09:52:27,10114.0,6,20000.00,30114.0,0.66,M06WAWY,Home Along Bagong Silang 1,Appliances,M06WAWY,null,null,Premium,null,null
2,ed2f1d46-c6d2-4e95-a091-6a395b76d55c,60823611090011,SIL-Instore,639265262881,2024-01-07 10:37:58,8239.0,9,2060.00,10299.0,0.20,MY7RZCA,Home Along Baliuag 2,Appliances,NaN,NaN,NaN,NaN,NaN,NaN
3,98c98986-d48d-42c5-a37e-15eff0c71700,60824858150011,SIL-Instore,639952664427,2024-04-28 10:59:16,8063.0,6,896.00,8959.0,0.10,M2Q79KY,Home Along Los Banos 2,Appliances,M2Q79KY,null,null,null,null,Starter
4,c47e9d9d-7f85-471e-b1a3-fbb3fe3888e6,60827313590015,SIL-Instore,639466847850,2024-08-07 18:39:06,12599.0,6,5400.00,17999.0,0.30,M8ZJKRN,Home Along Meycauayan 1 Sto Nino,Appliances,M8ZJKRN,Advanced,null,null,null,null


In [17]:
df.to_csv(r"SIL2024DisbursementLoan_Downpaymentwithstoretypeandsegment.csv", index = False)

In [18]:
sq ="""with b as 
(Select distinct lmt.digitalLoanAccountId, lmt.purpleKey,
lmt.startApplyDateTime,
  tbtc.loanCategoryId, tbpm.product_id, tbpm.product_description,  tqpd.segment_id, tsm.segment_name
, lmt.disbursementDateTime, lmt.new_loan_type
from `risk_credit_mis.loan_master_table` lmt 
inner join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc on tbtc.digitalLoanAccountId = lmt.digitalLoanAccountId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm on tbpm.product_id = tbtc.loanCategoryId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_qr_promoter_details tqpd on tqpd.digitalLoanAccountId = lmt.digitalLoanAccountId
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_segment_mtb tsm on cast(tsm.segment_id as numeric) = cast(tqpd.segment_id as numeric)
where tsm.segment_name is not null
)
select digitalLoanAccountId, purpleKey,  product_description from b"""

In [19]:
dfd = client.query(sq).to_dataframe()
dfd.head()

,digitalLoanAccountId,purpleKey,product_description
0,af155bfb-7b0e-4f5c-9ff0-beab964df919,MGFYCE0,"PC, Laptop or Game consoles"
1,19d80abe-0ed0-4bd3-89ef-62474b79b856,M7261RR,Gym or Fitness equipment
2,39c3d104-f13a-4099-a52f-4b710abf09db,MDLENQF,"PC, Laptop or Game consoles"
3,0c532517-5206-44b6-87a0-08abbce42629,M7261RR,"PC, Laptop or Game consoles"
4,0d100577-ec1e-4d37-8a11-22363614659e,MHQ2U64,"PC, Laptop or Game consoles"


In [20]:
tab1 = dfd.groupby(['purpleKey', 'product_description']).agg(countof=('digitalLoanAccountId', 'nunique')).reset_index()
tab2= tab1.pivot_table(index = 'purpleKey', columns='product_description', values='countof', aggfunc='sum').reset_index()

In [21]:
tab2.head()

product_description,purpleKey,Accessories or Peripherals,Air conditioners,Bed frames or Cribs,"Dining set, Chairs or Tables",Gym or Fitness equipment,Mobile Phones,"Oven, Cooking Range, Griller","PC, Laptop or Game consoles",Refrigerators,"Sala set, Couch, Sofa set",Small Appliances,"Stereo set, Soundbar, Audio equipment",Televisions,"Washing machine, dryer or both"
0,M01FEG9,2.0,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M02DYRA,5.0,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M03EKHG,1.0,51.0,4.0,16.0,NaN,68.0,4.0,1.0,46.0,24.0,8.0,21.0,33.0,40.0
3,M06W4RG,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,3.0,18.0
4,M06WAWY,4.0,71.0,30.0,14.0,NaN,97.0,6.0,NaN,75.0,33.0,22.0,33.0,126.0,90.0


In [22]:
tab2['goods_type'] = tab2['Mobile Phones'].apply(lambda x: 'Mobile' if x >= 1.0 else 'Appliance')

In [23]:
tab2.columns

Index(['purpleKey', 'Accessories or Peripherals', 'Air conditioners',
       'Bed frames or Cribs', 'Dining set, Chairs or Tables',
       'Gym or Fitness equipment', 'Mobile Phones',
       'Oven, Cooking Range, Griller', 'PC, Laptop or Game consoles',
       'Refrigerators', 'Sala set, Couch, Sofa set', 'Small Appliances',
       'Stereo set, Soundbar, Audio equipment', 'Televisions',
       'Washing machine, dryer or both', 'goods_type'],
      dtype='object', name='product_description')

In [24]:
tab3  = tab2[['purpleKey', 'goods_type']].copy()
tab3.head()

product_description,purpleKey,goods_type
0,M01FEG9,Mobile
1,M02DYRA,Mobile
2,M03EKHG,Mobile
3,M06W4RG,Appliance
4,M06WAWY,Mobile


In [25]:
tab3['goods_type'].value_counts()

goods_type
Mobile       713
Appliance    114
Name: count, dtype: int64

In [27]:
df.drop(columns = 'purpleKey_y', inplace = True)
df.rename(columns = {'purpleKey_x':'purpleKey'}, inplace = True)
df.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'new_loan_type',
       'mobileno', 'disbursementDateTime', 'disbursedLoanAmount',
       'disbursedLoanTenur', 'downPaymentAmount', 'Items_Sold_Price',
       'Downpayment_rate', 'purpleKey', 'merchantName', 'STORE_TAG',
       'Advanced', 'Basic', 'Premium', 'SIL-Zero', 'Starter'],
      dtype='object')

In [28]:
df = df.merge(tab3, on='purpleKey', how='left')
df.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'new_loan_type',
       'mobileno', 'disbursementDateTime', 'disbursedLoanAmount',
       'disbursedLoanTenur', 'downPaymentAmount', 'Items_Sold_Price',
       'Downpayment_rate', 'purpleKey', 'merchantName', 'STORE_TAG',
       'Advanced', 'Basic', 'Premium', 'SIL-Zero', 'Starter', 'goods_type'],
      dtype='object')

In [29]:
df.to_csv(r"SIL2024DisbursementLoan_Downpaymentwithstoretypeandsegmentwithgoodtype.csv", index = False)